## Imports

In [9]:
import cv2, os
import random,shutil
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.image import imread
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import models, layers, Model
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical

from tensorflow.keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.models import Sequential, load_model
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization

In [10]:
!ls /kaggle/input/drowsiness-detection

In [11]:
data_dir = '/kaggle/input/drowsiness-detection'

## Image Data Generator and Data Augmentation

In [14]:
def generator(dir, gen=image.ImageDataGenerator(rescale=1./255), shuffle=True,batch_size=1,target_size=(24,24),
              class_mode='categorical' ):

    return gen.flow_from_directory(dir,batch_size=batch_size,shuffle=shuffle,color_mode='grayscale',
                                   class_mode=class_mode,target_size=target_size)

batch= 32
target=(24,24)
train_batch = generator(data_dir,shuffle=True, batch_size=batch,target_size=target)
valid_batch = generator(data_dir,shuffle=True, batch_size=batch,target_size=target)
size_per_epoch= len(train_batch.classes)//batch
validation_size = len(valid_batch.classes)//batch
print(size_per_epoch,validation_size)

## Using Basic CNN Architecture

In [15]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
    #32 convolution filters used each of size 3x3 again
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
    #64 convolution filters used each of size 3x3 choose the best features via pooling
    #randomly turn neurons on and off to improve convergence
    Dropout(0.25),
    #flatten since too many dimensions, we only want a classification output
    Flatten(),
    #fully connected to get all relevant data
    Dense(128, activation='relu'),
    #one more dropout for convergence' sake :) 
    Dropout(0.5),
    #output a softmax to squash the matrix into output probabilities
    Dense(2, activation='softmax')
])
model.summary()

In [16]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_batch, validation_data=valid_batch,epochs=2,steps_per_epoch=size_per_epoch ,validation_steps=validation_size)
model.save('CNN_Drowsiness.h5')

## Using ResNet-50 Transfer Learning Technique

In [17]:
train_datagen = ImageDataGenerator(validation_split=0.3,
                                   preprocessing_function=preprocess_input) 

train_generator = train_datagen.flow_from_directory( data_dir,
                                                     target_size=(224,224),
                                                     batch_size=100,
                                                     shuffle=True,
                                                     class_mode='binary',
                                                     subset='training')

validation_datagen = ImageDataGenerator(validation_split=0.3,
                                        preprocessing_function=preprocess_input)

validation_generator =  validation_datagen.flow_from_directory( data_dir,
                                                                target_size=(224,224),
                                                                batch_size=100,
                                                                class_mode='binary',
                                                                subset='validation')  

In [18]:
model_res50 = Sequential()

model_res50.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet'
    ))

model_res50.add(Dense(1, activation='sigmoid'))
model_res50.layers[0].trainable = False 
model_res50.summary()

steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)

In [19]:
model_res50.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

history = model_res50.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    validation_steps=steps_per_epoch_validation,
    epochs=3,
    validation_data=validation_generator,
    verbose=1
)

In [20]:
model_res50.save('ResNet50_Drowsiness.h5')

In [21]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

# Retrieve a list of list results on training and test data sets for each training epoch
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()


# Plot training and validation loss per epoch
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.title('Training and validation loss')
plt.legend()

plt.show()